In [2]:
import duckdb  # noqa: F401


In [3]:
con = duckdb.connect(database='../data/dados_duckdb.db', read_only=False)

In [3]:
produtos = """
SELECT * 
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS ROW
    FROM bronze_produtos
    WHERE data_ingestao >= '2025-06-26'
) WHERE ROW = 1
"""

In [4]:
df = con.execute(produtos).fetch_df()
df

,NATBR,MARKTZ,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,ROW
0,10004,SERRA,BT50,100,200,z0019_2.csv,2025-06-26 22:28:29.688976,1
1,10003,PREGO,BT10,100,50,z0019_2.csv,2025-06-26 22:28:29.688976,1
2,10001,PARAFUSO,BT10,100,100,z0019_2.csv,2025-06-26 22:28:29.688976,1
3,10005,MACHAD,BT10,100,50,z0019_2.csv,2025-06-26 22:28:29.688976,1
4,10002,MARTELO,BT50,100,1500,z0019_2.csv,2025-06-26 22:28:29.688976,1


In [5]:
df_final = df.drop(columns=["nome_arquivo", "data_ingestao", 'ROW'])

In [6]:
df_final

,NATBR,MARKTZ,WERKS,MAINS,LABST
0,10004,SERRA,BT50,100,200
1,10003,PREGO,BT10,100,50
2,10001,PARAFUSO,BT10,100,100
3,10005,MACHAD,BT10,100,50
4,10002,MARTELO,BT50,100,1500


In [7]:
df_final = df_final.rename(
        columns={
            'NATBR': 'id', 'MARKTZ': 'nm_produto', 'WERKS': 'id_categoria',
            'MAINS': 'id_fornecedor', 'LABST':'vl_preco'
            }
    )

In [8]:
df_final

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200
1,10003,PREGO,BT10,100,50
2,10001,PARAFUSO,BT10,100,100
3,10005,MACHAD,BT10,100,50
4,10002,MARTELO,BT50,100,1500


In [9]:
df2 = df_final
df2

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200
1,10003,PREGO,BT10,100,50
2,10001,PARAFUSO,BT10,100,100
3,10005,MACHAD,BT10,100,50
4,10002,MARTELO,BT50,100,1500


In [10]:
df_final.dtypes

id               object
nm_produto       object
id_categoria     object
id_fornecedor    object
vl_preco         object
dtype: object

In [11]:
df2 = df2.astype(
    {
        "id": int,
        "nm_produto": str,
        "id_categoria": str,
        "id_fornecedor": int,
        "vl_preco": float,
    }
)


In [12]:
df2.dtypes

id                 int32
nm_produto        object
id_categoria      object
id_fornecedor      int32
vl_preco         float64
dtype: object

In [13]:
df2

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200.0
1,10003,PREGO,BT10,100,50.0
2,10001,PARAFUSO,BT10,100,100.0
3,10005,MACHAD,BT10,100,50.0
4,10002,MARTELO,BT50,100,1500.0


In [ ]:
tabela = """ 
CREATE TABLE IF NOT EXISTS produtos (
        id BIGINT,
        nm_produto TEXT,
        id_categoria TEXT,
        id_fornecedor BIGINT,
        vl_preco FLOAT
)
"""

In [19]:
con.execute(tabela)

In [ ]:
con.execute("INSERT INTO  produtos SELECT * FROM df2")

In [5]:
df_resultado  = con.execute("SELECT * FROM produtos").fetch_df()
df_resultado

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200.0
1,10003,PREGO,BT10,100,50.0
2,10001,PARAFUSO,BT10,100,100.0
3,10005,MACHAD,BT10,100,50.0
4,10002,MARTELO,BT50,100,1500.0


In [6]:
con.close()